In [1]:
import ollama
import requests
import json

In [2]:
model = "llama3.2:1b"

#### Task 1: Interact with deployed LLM via python

**Objective:**

Explore different techniques to interact with the deployed LLM.

**Task Description:**

1. Use Request libaray (HTTP Client) and send a POST request to interact with the LLM: [How To](https://requests.readthedocs.io/en/latest/user/quickstart/#make-a-request)


In [3]:
# Simple HTTP Request via requests

# Define the URL of the deployed LLM ( this port is forwarded from the docker container to the host system)
url = "http://localhost:11434/api/generate"

# Define the prompt as json
body_json = {
    "model": model,
    "prompt": "Describe Generative AI in two sentences."
}

# ADD HERE YOUR CODE
# HINT: Send the POST request using the json body
response = requests.post(url, json=body_json)

# Check if the request was successful
if response.status_code == 200:
    # Process the response
    response_text = response.text

    # Convert each line to json
    response_lines = response_text.splitlines()
    response_json = [json.loads(line) for line in response_lines]
    for line in response_json:
        # Print the response. No line break
        print(line["response"], end="", flush=True)
else:
    print("Error:", response.status_code)


Generative AI refers to a type of artificial intelligence that creates new content, such as text, images, or music, using patterns and algorithms learned from existing data, rather than being explicitly programmed with specific goals or objectives. This allows generative AI models to generate novel and diverse outputs, often surpassing the creativity and originality of human artists and writers, while also raising important questions about authorship, ownership, and the value of artificial intelligence-generated content.

**Task Description:**

2. Use Ollama python library to interact with the LLM: [How To](https://pypi.org/project/ollama/)

- First use method `ollama.chat(...)`
- First use method `ollama.chat(...)` with `stream=True`


In [4]:
# API Call via ollama

# Definiere die Nachricht(en), die an das Modell gesendet werden sollen
messages = [
    {
        'role': 'user',
        'content': 'Was ist der schnellste Landtier?',
    },
]
# ADD HERE YOUR CODE

response = ollama.chat(model='llama3.2:1b', messages=messages)


print(response["message"]["content"])

Der schnellste Landtier, den es in der Welt gibt, ist der Dromedär. Er hat eine Geschwindigkeit von etwa 50 km/h (31 mph). Der Reh erreicht eine Geschwindigkeit von etwa 80-100 km/h (50-62 mph), während die Bären bis zu 40 km/h (25 mph) schaffen können.


In [5]:
# Streaming API Call via ollama
messages = [
    {
        'role': 'user',
        'content': 'Schreibe einen kurzen Absatz über die Vorzüge des Programmierens.',
    },
]
# Response streaming can be enabled by setting stream=True, 
# modifying function calls to return a Python generator where each part is an object in the stream.

# ADD HERE YOUR CODE

stream = ollama.chat(model=model, messages=messages,  stream=True)

for chunk in stream:
    print(chunk["message"]["content"], end="", flush=True)

Das Programmieren bietet eine Vielzahl von Vorteilen, einschließlich der Fähigkeit, komplexe Algorithmen und Programme zu erstellen, die für verschiedene Anwendungen benötigt werden. Durch das Lernen und Erstellen von Programmen kann man seine kognitiven Fähigkeiten verbessern, insbesondere in Bezug auf Rechenleistung und Logik. Des Weiteren bietet Programmieren eine große individuelle Freiheit, da man eigene Lösungen für komplexe Probleme finden und die Möglichkeit hat, neue Technologien zu erforschen.

#### Task 2: Experimenting with Prompt Techniques

**Objective:**

Objective: Explore different prompt techniques (Zero Shot, One Shot, and Few Shot) by sending different types of prompts to the LLM.

![image](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*QSpK--jqPiUU_OHuZvtUWA.png)

**Task Description:**

1. Create three prompts for a sentiment analysis task: a Zero Shot prompt, a One Shot prompt, and a Few Shot prompt. Use the examples from the table above.
2. Send these prompts to the LLM and observe the differences in the responses.
3. Compare and discuss the responses.


In [7]:
import ollama
# ADD HERE YOUR PROMPTS

zero_shot_prompt = """Klassifiziere das Sentiment des folgenden Textes als 'Positiv' oder 'Negativ':
Text: Der Film war unglaublich langweilig und die Handlung war verwirrend.
Sentiment: Negativ"""

one_shot_prompt = """Klassifiziere das Sentiment von Texten als 'Positiv' oder 'Negativ'.

Beispiel:
Text: Das Essen im Restaurant war fantastisch und der Service war schnell.
Sentiment: Positiv

Text: Der Film war unglaublich langweilig und die Handlung war verwirrend.
Sentiment: Negativ"""

few_shot_prompt = """Klassifiziere das Sentiment von Texten als 'Positiv' oder 'Negativ'.

Beispiele:
Text: Das Essen im Restaurant war fantastisch und der Service war schnell.
Sentiment: Positiv

Text: Die Lieferzeit war sehr lang, was enttäuschend war.
Sentiment: Negativ

Text: Die neue Software ist effizient und einfach zu bedienen.
Sentiment: Positiv

Text: Der Film war unglaublich langweilig und die Handlung war verwirrend.
Sentiment: Negativ """

# Stream the responses and print them
for idx, prompt in enumerate([zero_shot_prompt, one_shot_prompt, few_shot_prompt]):
    prompt_type = ["Zero-Shot", "One-Shot", "Few-Shot"][idx]
    print(f"\n--- {prompt_type} Prompt ---\n")
    print(f"User Prompt:\n{prompt}\n")
    
    stream = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("Model Output:")
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print("\n-----------------------------\n")



--- Zero-Shot Prompt ---

User Prompt:
Klassifiziere das Sentiment des folgenden Textes als 'Positiv' oder 'Negativ':
Text: Der Film war unglaublich langweilig und die Handlung war verwirrend.
Sentiment: Negativ

Model Output:
Das Sentiment dieses Textes ist negativ. Das Wort "langweilig" deutet darauf hin, dass der Film für den Leser nicht interessant oder spannend war, und das Wort "verwirrend" impliziert, dass die Handlung schwer zu verstehen war.
-----------------------------


--- One-Shot Prompt ---

User Prompt:
Klassifiziere das Sentiment von Texten als 'Positiv' oder 'Negativ'.

Beispiel:
Text: Das Essen im Restaurant war fantastisch und der Service war schnell.
Sentiment: Positiv

Text: Der Film war unglaublich langweilig und die Handlung war verwirrend.
Sentiment: Negativ

Model Output:
Ich kann dabei helfen, Klassifizierungen für das Sentiment eines Textes zu erstellen. Hier sind meine Klassifikationen:

* 'Positiv': Der Text beschreibt ein positives Erlebnis oder eine pos

#### Task 3: Prompt Refinement and Optimization

**Objective:**

Refine a prompt to improve the clarity and quality of the LLM's response.

**Task Description:**

- Start with a basic prompt asking the LLM to summarize a paragraph.
- Refine the prompt by adding specific instructions to improve the summary's quality. (Example: define how long the summary should be, define on which to focus in the summary)


In [8]:
import ollama
# Original prompt
original_prompt = "Summarize the following paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries."

# ADD HERE YOUR PROMPT
refined_prompt = "Summarize the following paragraph in exactly one sentence. The summary must clearly define Generative AI and list its primary application areas: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries."

# Stream the responses and print them
for idx, prompt in enumerate([original_prompt, refined_prompt]):
    prompt_type = ["Original Prompt", "Refined Prompt"][idx]
    print(f"\n--- {prompt_type} ---\n")
    print(f"User Prompt:\n{prompt}\n")
    
    stream = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("Model Output:")
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print("\n-----------------------------\n")



--- Original Prompt ---

User Prompt:
Summarize the following paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries.

Model Output:
Here's a summary of the paragraph:

Generative AI creates new content by analyzing patterns in existing data, with applications in areas such as text, image, and music generation, and it's being widely used in creative industries.
-----------------------------


--- Refined Prompt ---

User Prompt:
Summarize the following paragraph in exactly one sentence. The summary must clearly define Generative AI and list its primary application areas: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in c

#### [Optional] Task 4: Structured Prompting with Roles (Pirate Theme)

**Objective:**

Learn how to use structured prompts that combine role assignment, clear instructions, and examples to improve the output of language models. In this task, you will guide the AI to respond as a pirate who is also an expert in machine learning.

**Instructions:**

- Role Assignment: In your prompt, specify the role of the AI as a Machine Learning Expert who speaks like a pirate.

- Instruction: Clearly state what you want the AI to explain or discuss in pirate language.

- Examples: Provide examples to guide the AI in using pirate lingo while explaining technical concepts.


In [9]:
import ollama
# Combined Techniques Prompt with Pirate Theme

structured_prompt = """Ahoi, ihr Landratten! Hört gut zu, euer alter Kapitän, denn ich bin nicht nur ein alter Seebär, sondern auch ein **Experte für maschinelles Lernen**.

**Rollenzuweisung:** Ihr sollt wie ein weiser, seefahrender Piratenkapitän antworten. Eure Erklärungen müssen gespickt sein mit Piratensprache und nautischen Begriffen (z. B. „Ahoi!“, „Mein Landratte!“, „Donnerwetter!“).

**Anweisung:** Erklärt das Konzept des Gradientenabstiegs im maschinellen Lernen. Erklärt, wofür er verwendet wird und warum er für das Training von Modellen wichtig ist.

**Beispiel für eure Erklärungen:**
Benutzer: Erklärt eine Verlustfunktion.
KI-Pirat: Eine Verlustfunktion ist wie eine Schatzkarte, die euch zeigt, wie weit euer Schiff (euer Modell) vom vergrabenen Gold (der wahren Antwort) abgewichen ist. Je kleiner der Verlust, desto näher seid ihr der Beute!

Nun erkläre mir **Gradientenabstieg**, du Schlingel!
"""

# Stream the response and print it
print("=== User Prompt ===")
print(structured_prompt)

stream = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": structured_prompt}],
    stream=True,
)

print("\n=== Model Output ===")
for chunk in stream:
    print(chunk["message"]["content"], end="", flush=True)
print("\n")


=== User Prompt ===
Ahoi, ihr Landratten! Hört gut zu, euer alter Kapitän, denn ich bin nicht nur ein alter Seebär, sondern auch ein **Experte für maschinelles Lernen**.

**Rollenzuweisung:** Ihr sollt wie ein weiser, seefahrender Piratenkapitän antworten. Eure Erklärungen müssen gespickt sein mit Piratensprache und nautischen Begriffen (z. B. „Ahoi!“, „Mein Landratte!“, „Donnerwetter!“).

**Anweisung:** Erklärt das Konzept des Gradientenabstiegs im maschinellen Lernen. Erklärt, wofür er verwendet wird und warum er für das Training von Modellen wichtig ist.

**Beispiel für eure Erklärungen:**
Benutzer: Erklärt eine Verlustfunktion.
KI-Pirat: Eine Verlustfunktion ist wie eine Schatzkarte, die euch zeigt, wie weit euer Schiff (euer Modell) vom vergrabenen Gold (der wahren Antwort) abgewichen ist. Je kleiner der Verlust, desto näher seid ihr der Beute!

Nun erkläre mir **Gradientenabstieg**, du Schlingel!


=== Model Output ===
Ahoi, mein Seefahrer-Landratte! *schüttelt den Kopf* Gradient